In [46]:
import os
import pandas as pd
import numpy as np

In [47]:
df_movie_names = pd.read_csv('./data/ml-latest-small/movies.csv')
df_ratings = pd.read_csv('./data/ml-latest-small/ratings.csv')

### Check unique values

In [48]:
df_ratings.shape, df_ratings['movieId'].nunique(),df_movie_names.shape, df_movie_names['title'].nunique()
# Why less unique titles? -> some movies have more than one movieId
# movieId is NOT UNIQUE

((100836, 4), 9724, (9742, 3), 9737)

### Create unique movieId column in ratings_df and movie_title_df

#### Find duplicate names/ movies that are assigned to more than one movieId

In [49]:
duplicateRowsDF = df_movie_names[df_movie_names.duplicated(subset=['title'])]
duplicateRowsDF

,movieId,title,genres
5601,26958,Emma (1996),Romance
6932,64997,War of the Worlds (2005),Action|Sci-Fi
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
9135,147002,Eros (2004),Drama|Romance
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller


In [50]:
dup_titles = duplicateRowsDF['title'].tolist()
dup_titles

['Emma (1996)',
 'War of the Worlds (2005)',
 'Confessions of a Dangerous Mind (2002)',
 'Eros (2004)',
 'Saturn 3 (1980)']

In [51]:
dup = df_movie_names[df_movie_names['title'].isin(dup_titles)]
dup.sort_values(by='title', ascending=True)

,movieId,title,genres
4169,6003,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Thriller
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
650,838,Emma (1996),Comedy|Drama|Romance
5601,26958,Emma (1996),Romance
5854,32600,Eros (2004),Drama
9135,147002,Eros (2004),Drama|Romance
2141,2851,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller
5931,34048,War of the Worlds (2005),Action|Adventure|Sci-Fi|Thriller
6932,64997,War of the Worlds (2005),Action|Sci-Fi


In [52]:
# create list out of these movieIds:
dup_ids = dup['movieId'].tolist()
dup_ids

[838, 2851, 6003, 26958, 32600, 34048, 64997, 144606, 147002, 168358]

In [53]:
# check which movieId these titles have in the ratings df:
dup_in_rat = df_ratings[df_ratings['movieId'].isin(dup_ids)]
dup_in_rat['movieId'].unique()
# this dataframe also has the different movie Ids for the same films 

array([  2851,    838,  34048,  64997,   6003, 144606, 147002,  26958,
       168358,  32600])

#### change the movieIds that occur two times to the first occuring movieId

In [54]:
# in ratings_df & movie_name df

# create movieId_unique column 
# 1. copy the existing ids
df_ratings['movieId_unique'] = df_ratings['movieId'] 
df_movie_names['movieId_unique'] = df_movie_names['movieId']

# create a dictionary with unique movieIds for the films (take the lower number):
movie_id_dict = {144606: 6003, 
                 26958: 838, 
                 147002: 32600,
                 168358: 2851,
                 64997: 34048   } 

# Remap the values of the dataframe 
df_ratings = df_ratings.replace({"movieId_unique": movie_id_dict}) 
df_movie_names = df_movie_names.replace({"movieId_unique": movie_id_dict}) 

# test 1:
df_ratings[df_ratings['movieId']==64997]

,userId,movieId,rating,timestamp,movieId_unique
4747,28,64997,3.5,1234850075,34048
11451,68,64997,2.5,1230497715,34048


In [55]:
# test 2: 
df_movie_names[df_movie_names['movieId']==64997]

,movieId,title,genres,movieId_unique
6932,64997,War of the Worlds (2005),Action|Sci-Fi,34048


In [56]:
df_ratings['movieId_unique'].nunique(), df_movie_names['movieId_unique'].nunique()
# there are more movie names than rated movies !

(9719, 9737)

### -> some movies do not have a rating!

### left-merge ratings_df and movie_names_df

In [57]:
df_ratings.head()

,userId,movieId,rating,timestamp,movieId_unique
0,1,1,4.0,964982703,1
1,1,3,4.0,964981247,3
2,1,6,4.0,964982224,6
3,1,47,5.0,964983815,47
4,1,50,5.0,964982931,50


In [58]:
# left merge to keep only movies with existing ratings
df = pd.merge(df_ratings, df_movie_names, on='movieId_unique', how='left')
# check for number of unique ids
df['movieId_unique'].nunique()

9719

In [59]:
df.head()

,userId,movieId_x,rating,timestamp,movieId_unique,movieId_y,title,genres
0,1,1,4.0,964982703,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,3,3,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,6,6,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,47,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,50,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


###  construct a dictionary for name: id 

In [61]:
# To map movie names to movie ID’s, construct a {name: id} dictionary
title_dict = dict(zip(df.title, df.movieId_unique))
len(title_dict)

9719

---